In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import sys
sys.path.append('..')
import pytpc
from pytpc.constants import *
from scipy.ndimage.filters import gaussian_filter
from math import sin, cos

In [ ]:
def make_event(pos, en, clock, vd, ioniz, proj_mass, offset, shapetime):
    
    # Find energy deposition for each point
    de = np.abs(np.diff(en))
    ne = np.round(de*1e6*proj_mass / ioniz)  # last factor after mass is gain
    ne = np.insert(ne, 0, 0)  # indexed by simulated point number
    
    # Uncalibrate the position data
    uncal = pytpc.evtdata.uncalibrate(pos, vd, clock)
    tbs = uncal[:, -1]
    print(tbs)
    
    # Find the pad for each point
    pca = np.round(np.array([pytpc.simulation.find_pad_coords(p[0], p[1]) for p in uncal[:, 0:2]]))
    pnums = np.array([pdict[tuple(a)] for a in pca])
    unique_pads = np.unique(pnums)
    
    shapetime_tb = shapetime*1e-9 * clock*1e6
    
    # Build the event
    evt = pytpc.Event()
    evt.traces = np.zeros(unique_pads.shape, dtype=evt.dt)
    for i, p in enumerate(unique_pads):
        idxs = np.where(pnums == p)
        tr = np.zeros(512)
        for t, v in zip(tbs[idxs], ne[idxs]):
            tr[t] += v
        evt.traces[i]['pad'] = p
        evt.traces[i]['data'][:] = gaussian_filter(tr, shapetime_tb, mode='constant')
    return evt

In [ ]:
pads = pytpc.generate_pad_plane()
keys = tuple(map(tuple, np.round(pads.mean(1)).tolist()))
pdict = {k:i for i, k in enumerate(keys)}

In [ ]:
tilt = 7*degrees
clock = 6.25  # MHz
emag = 15e3
bmag = 0.5691
ef = np.array([0, 0, emag])
bfield_orig = np.array([0, 0, bmag])
trans = np.array([[1, 0, 0],
                  [0, cos(-tilt), -sin(-tilt)],
                  [0, sin(-tilt), cos(-tilt)]])
bf = np.dot(trans, bfield_orig)
vd = pytpc.simulation.drift_velocity_vector(-3.32, emag, bmag, tilt)

In [ ]:
pt = pytpc.Particle(4, 2, 2.2, position=(0, 0, 1.), azimuth=pi/2, polar=pi+tilt)
he = pytpc.gases.HeliumGas(150.)
simres = pytpc.track(pt, he, ef, bf)
simres['pos'] *= 1000
# simres['pos'][:, 2] = 1000 - simres['pos'][:, 2]

In [ ]:
evt = make_event(simres['pos'], simres['en'], clock, vd, 40.2, pt.mass_num, 100, 334.)

In [ ]:
plt.plot(simres['pos'][:, 2])

In [ ]:
pytpc.pad_plot(evt.hits());

In [ ]:
plt.plot(evt.traces['data'].sum(0))

In [ ]:
plt.plot(evt.traces['data'].sum(0), 'k--')
for x in evt.traces['data']:
    plt.plot(x)

In [ ]:
pytpc.chamber_plot(evt.xyzs(vd, clock), zscale='dist');

In [ ]:
plt.plot(simres['pos'][:, 0], simres['pos'][:, 1])
plt.xlim(-275, 275)
plt.ylim(-275, 275)
plt.gca().set_aspect('equal')

In [ ]:
plt.plot(simres['pos'][:, 2], simres['pos'][:, 1])
plt.xlim(0, 1000)
plt.ylim(-275, 275)
plt.gca().set_aspect('equal')

In [ ]:
uncal = pytpc.evtdata.uncalibrate(simres['pos'], vd, clock)

In [ ]:
plt.plot(uncal[:, 0], uncal[:, 1])
plt.xlim(-275, 275)
plt.ylim(-275, 275)
plt.gca().set_aspect('equal')

In [ ]:
plt.plot(uncal[:, 2], uncal[:, 1])
plt.xlim(0, 512)
plt.ylim(-275, 275)
plt.gca().set_aspect('equal')